In [110]:
#! /usr/bin/python3.5

In [111]:
a=["A","B","C","D","hh"]

In [117]:
def chunkIt(seq, num):
   
    avg = len(seq) / float(num)
    out = []
    last = 0.0
    if num > 1:
        while last < len(seq):
            out.append(seq[int(last):int(last + avg)])
            last += avg
    else:
        return seq

    return out

In [118]:
def model(a_list,i):
    print("list model",a_list,i)

In [144]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
    #return [x.name for x in local_device_protos]

In [153]:

import os
import GPUtil as GPU
import tensorflow as tf
#import threading
import timeit
import time
#from tensorflow import logging
from tensorflow.python.client import timeline
#os.environ['TF_CPP_MIN_VLOG_LEVEL']='3'
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
#logging.set_verbosity(logging.INFO)
#logging.vlog(1, "Registering %s (%s) in %s.", name, candidate, self._name)

config = tf.ConfigProto(device_count={"GPU": 2,"CPU":1},
                        allow_soft_placement=True,
                        inter_op_parallelism_threads=16,
                        intra_op_parallelism_threads=16,
                        use_per_session_threads=True,
                        log_device_placement=True)
config.gpu_options.allow_growth = True
import setGPU
print("setGPU.stats",setGPU.stats)
print("setGPU.stats",setGPU.bestGPU)
print("setGPU.ratios",setGPU.ratios)
print("setGPU.ids",setGPU.ids)
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    GPU_LIST=[]
    list_file_gpu=[]
    import os
    #os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"
    with tf.device('/cpu'):
        print("Start Time CPU",time.time(), time.clock())
        start = timeit.default_timer()
        GPU_LIST=get_available_gpus()
        list_file_gpu=chunkIt(a,len(GPU_LIST))
        print(list_file_gpu)
        for i in range(len(GPU_LIST)):
            with tf.device(tf.DeviceSpec(device_type="GPU", device_index=i)):
                start_time=time.time()
                start_oclock=time.clock()
                print("Start Time GPU",i,"Time is ",start_time,start_oclock)
                #gpu_full_process(file_names,path_data_base,path_tf,path_non_redundant,path_idf,file_path_sim)
                #Your statements here
                print("In GPU Dev",i)
                model(list_file_gpu[i],i)
                end_time=time.time()
                end_oclock=time.clock()
                print("End Time GPU",i,"Time is ",end_time,end_oclock)
                total_time_GPU=end_time-start_time
                total_clock_GPU=end_oclock-start_oclock
                print("total_time_GPU",total_time_GPU)
                print("total_clock_GPU",total_clock_GPU)
        stop = timeit.default_timer()
        print("End Time",time.time(), time.clock())
        GPU.showUtilization(all=True)#https://github.com/anderskm/gputil
        tl = timeline.Timeline(run_metadata.step_stats)
        print(tl.generate_chrome_trace_format(show_memory=True))

setGPU.stats GPUStatCollection(host=fsg-Satellite-P750, [
  [0] GeForce GT 540M  | 63'C,  ?? % |   279 /   964 MB | (Not Supported)
])
setGPU.stats 0
setGPU.ratios <map object at 0x7fb36d46f4e0>
setGPU.ids <map object at 0x7fb36d442fd0>
Start Time CPU 1512480856.8311646 30.410207
['A', 'B', 'C', 'D', 'hh']
Start Time GPU 0 Time is  1512480856.8318973 30.411237
In GPU Dev 0
list model A 0
End Time GPU 0 Time is  1512480856.8332317 30.413104
total_time_GPU 0.0013344287872314453
total_clock_GPU 0.0018670000000007292
End Time 1512480856.833459 30.413492


ValueError: could not convert string to float: '[Not Supported]'

In [133]:
import json, socket, subprocess, sys, time
try:
    delay = int(sys.argv[1])
except:
    delay = 1

try:
    count = int(sys.argv[2])
except:
    count = None

i = 0
while True:
    GPU = socket.gethostname() + ":0[gpu:0]"
    print('GPU',GPU)
    cmd = ['nvidia-settings', '-t', '-q', GPU + '/GPUUtilization']
    print('cmd',cmd)
    gpu_util = subprocess.check_output(cmd).strip().split(",")
    print("gpu_util",gpu_util)
    gpu_util = dict([f.strip().split("=") for f in gpu_util])
    print("gpu_util",gpu_util)
    cmd[-1] = GPU + '/UsedDedicatedGPUMemory'
    print("cmd[-1]",cmd[-1])
    gpu_used_mem = subprocess.check_output(cmd).strip()
    print("gpu_used_mem",gpu_used_mem)
    print ("json",json.dumps({"used_mem": gpu_used_mem, "util": gpu_util, "time": int(time.time())}))
    if count != None:
        i += 1
        if i == count:
            exit(0)
            time.sleep(delay)

CalledProcessError: Command '['nvidia-settings', '-t', '-q', 'fsg-Satellite-P750:0[gpu:0]/GPUUtilization']' returned non-zero exit status 1.

In [2]:
# install memory util
import urllib.request
response = urllib.request.urlopen("https://raw.githubusercontent.com/yaroslavvb/memory_util/master/memory_util.py")
open("memory_util.py", "wb").write(response.read())

import memory_util
memory_util.vlog(1)
import tensorflow as tf

sess = tf.Session()
a = tf.random_uniform((1000,))
b = tf.random_uniform((1000,))
c = a + b
with memory_util.capture_stderr() as stderr:
    sess.run(c.op)
memory_util.print_memory_timeline(stderr, ignore_less_than_bytes=1000)


AssertionError: 2017-12-05 16:20:28.629512: I tensorflow/core/framework/log_memory.cc:35] __LOG_MEMORY__ MemoryLogTensorDeallocation { allocator_name: "cpu" }

In [6]:
import os

cmd = 'nvprof --query-metrics'

print(os.system(cmd))

0


In [6]:
# install memory_probe_ops

import urllib.request
import sys
#sys.path
#sys.path.append('./home/fsg/memory_probe_ops.so')

path = '/home/fsg/memory_probe_ops.so'
#os.environ['PATH'] += ':'+path
#print(os.environ)
#response = urllib.request.urlopen("https://github.com/yaroslavvb/memory_probe_ops/raw/master/linux.memory_probe_ops.so")
#open("memory_probe_ops.so", "wb").write(response.read())

import tensorflow as tf
from pathlib import Path

my_file = Path(path)
if my_file.exists():
    print("yes")
    
memory_probe_ops = tf.load_op_library(path)
print("Memory usage: ")
sess = tf.Session()
print(sess.run(memory_probe_ops.bytes_in_use()))


yes


NotFoundError: /home/fsg/memory_probe_ops.so: undefined symbol: _ZTIN10tensorflow8OpKernelE

In [32]:
# install memory util
import urllib.request
response = urllib.request.urlopen("https://raw.githubusercontent.com/yaroslavvb/memory_util/master/memory_util.py")
open("memory_util.py", "wb").write(response.read())

import memory_util
memory_util.vlog(1)
import tensorflow as tf

sess = tf.Session()
a = tf.random_uniform((1000,))
b = tf.random_uniform((1000,))
c = a + b
with memory_util.capture_stderr() as stderr:
    sess.run(c.op)
memory_util.print_memory_timeline(stderr, ignore_less_than_bytes=1000)


AssertionError: 2017-12-05 19:47:42.419361: I tensorflow/core/framework/log_memory.cc:35] __LOG_MEMORY__ MemoryLogTensorDeallocation { allocator_name: "cpu" }

In [34]:
import tensorflow as tf
import numpy as np
import os
import sys

so_name = "memory_probe_ops.so"

paths_to_search = []
paths_to_search.append('.')
for path in paths_to_search:
    full_so_name = os.path.join(path, so_name)
    if os.path.exists(full_so_name):
        print("Loading %s" %(full_so_name))
        memory_probe_ops = tf.load_op_library(full_so_name)
        break
else:
    print("Unable to find %s in %s, quitting"%(so_name, paths_to_search,))
    sys.exit()
    
run_with_tracing = False
config = tf.ConfigProto()
#config = tf.ConfigProto(graph_options=tf.GraphOptions(optimizer_options=tf.OptimizerOptions(opt_level=tf.OptimizerOptions.L0)))

for d in ["/cpu:0", "/gpu:0"]:
    with tf.device(d):
        sess = tf.InteractiveSession(config=config)
        mbs = 12
        n = mbs*250000
        inputs = tf.random_uniform((n,))
        print("Allocating %d MB variable on %s"%(mbs, d,))
        var = tf.Variable(inputs)
        probe_op = memory_probe_ops.bytes_in_use()
        max_op = memory_probe_ops.bytes_limit()
        name_op = memory_probe_ops.allocator_name()
        run_metadata = tf.RunMetadata()
        run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        print("Before init %10d out of %10d bytes with allocator %s" % tuple(sess.run([probe_op, max_op, name_op])))
        sess.run(var.initializer)
        print("After  init %10d out of %10d bytes with allocator %s" % tuple(sess.run([probe_op, max_op, name_op])))
        if run_with_tracing:
            print(run_metadata)

    

Loading ./memory_probe_ops.so


NotFoundError: ./memory_probe_ops.so: undefined symbol: _ZTIN10tensorflow8OpKernelE

In [9]:
import subprocess, re, os, sys
def run_command(cmd):
    """Run command, return output as string."""
    
    output = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True).communicate()[0]
    return output.decode("ascii")


In [18]:
def list_available_gpus():
    """Returns list of available GPU ids."""
    
    output = run_command("nvidia-smi -L")
    # lines of the form GPU 0: TITAN X
    gpu_regex = re.compile(r"GPU (?P<gpu_id>\d+):")
    result = []
    for line in output.strip().split("\n"):
        m = gpu_regex.match(line)
        assert m, "Couldnt parse "+line
        result.append(int(m.group("gpu_id")))
    return result

In [19]:
def gpu_memory_map():
    """Returns map of GPU id to memory allocated on that GPU."""

    output = run_command("nvidia-smi")
    gpu_output = output[output.find("GPU Memory"):]
    # lines of the form
    # |    0      8734    C   python                                       11705MiB |
    memory_regex = re.compile(r"[|]\s+?(?P<gpu_id>\d+)\D+?(?P<pid>\d+).+[ ](?P<gpu_memory>\d+)MiB")
    rows = gpu_output.split("\n")
    result = {gpu_id: 0 for gpu_id in list_available_gpus()}
    for row in gpu_output.split("\n"):
        m = memory_regex.search(row)
        if not m:
            continue
        gpu_id = int(m.group("gpu_id"))
        gpu_memory = int(m.group("gpu_memory"))
        result[gpu_id] += gpu_memory
    return result

In [20]:
gpu_memory_map()

{0: 0}